# NORTA
This is a homework implementation of the well-knwon algorithm to generation d-dimensional random vector with correlations. The NORTA or NORmal To Anything Algorithm can create this random vectors by giving just the d-marginal of each dimension. But it degenerates as the dimension grows.

The original paper can be found [here](https://pdfs.semanticscholar.org/8a45/34b1031d611876e2ee047ff5d2dc3757397c.pdf).

In [ ]:
import numpy as np
import scipy as sp
from scipy.stats import norm
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def gen_NORTA_from_rand(m,n,v,r):
    """
    weights \in [1,v]
    prices \in [r,v+r]
    """
    matriz = np.random.rand(m,n)
    R = np.corrcoef(matriz)
    print("Correlation Matriz:\n",R)
    L = np.linalg.cholesky(R) #lower
    W = np.random.normal(0, 1, (m,n))
    Z = np.dot(L,W)
    #print(np.shape(Z))
    X = np.empty([m, n])
    #print(np.shape(X))
    for i in range(n):
            X[0,i] = int(float(v-1)*norm.cdf(Z[0,i])+float(1.0))
            X[1,i] = int(float(v)*norm.cdf(Z[1,i])+float(r))
    #print(X)
    return X

In [ ]:
X = gen_NORTA_from_rand(2,1000,100,10)
plt.scatter(X[0,:],X[1,:])
plt.title("Plot of Dimension's relation")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()

In [ ]:
def gen_instances_from_NORTA(m,n,R,v,r):
    """
    m: dimensiones
    n: número de muestras
    R: matriz de correlaciones deseada
    weights \in [1,v]
    prices \in [r,v+r]
    """
    L = np.linalg.cholesky(R) #lower
    W = np.random.normal(0, 1, (m,n))
    Z = np.dot(L,W)
    X = np.empty([m, n])
    #print(np.shape(X))
    for i in range(n):
            X[0,i] = int(float(v-1)*norm.cdf(Z[0,i])+float(1.0))
            X[1,i] = int(float(v)*norm.cdf(Z[1,i])+float(r))
    return X

In [ ]:
corr = 0.7
R = [[1, corr],[corr, 1]]
print("Correlation Matriz:\n",R)
Z = gen_instances_from_NORTA(2,1000,R,100,10)
plt.scatter(Z[0,:],Z[1,:])
plt.title("Plot of Dimension's relation")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()

In [40]:
import numpy as np
from scipy.stats import norm, beta, expon, uniform, gamma, erlang

class Norta:
    """
    This class contains the NORmal To Anything (NORTA) algorithm to produce vector with correlations in the dimensions.
    
    Init:
    matrix M (mxn), wich has n-dimensions and m-observations per dimension.
    *args Marginal functions from which the dimension come from.
    If needed **kwargs will contain the parameter (by name) of the Marginal functions
    
    Methods:
    generate: Once initialize the parameters and functions, given the n-observation to generate, the method
    creates a matrix of mxn elementes, with identical Marginal distributions as the original described and with 
    the same correlation.
    """
    def __init__(self):
        self.contenedor = []
    
    def set_marginal(self, *args, **kwargs):
        print(args[0])
        if args[0] == 'normal':
            if ('mu' in kwargs) and ('sigma' in kwargs):
                self.contenedor.append({'function':norm,'mu':kwargs.get('mu'),'sigma':kwargs.get('sigma')})
                print("Function successfully added\n")
            else:
                print("Error when adding function to Norta")
                print("Remember to add a normal function toadd as arg and kwargs: set_marginal('normal',mu=,sigma=)")
        elif args[0] == 'beta':
            pass
        elif args[0] == 'expon':
            pass
        elif args[0] == 'uniform':
            if ('a' in kwargs) and ('b' in kwargs):
                self.contenedor.append({'function':uniform,'a':kwargs.get('a'),'b':kwargs.get('b')})
                print("Function successfully added\n")
            else:
                print("Error when adding function to Norta")
                print("Remember to add a normal function toadd as arg and kwargs: set_marginal('normal',a=,b=")
        elif args[0] == 'gamma':
            pass
        elif args[0] == 'erlang':
            pass
        else:
            print("Function does not exist in this implementation")
        
    def delete_marginal(self,*args):
        """
        Pronto
        """
        pass
    
    def set_data(self,data):
        """
        Recieves for now just a numpy array, that has to be the same dimension as the functions loaded so far
        """
        m,n = np.shape(data)
        if len(self.contenedor) != n:
            raise Exception('Not maching number in functions %d and dimension %d in Matrix' 
                            %(len(self.contenedor),m))
        else:
            self._generate_correlation_matrix(data,porcentaje=0.5,m=1000)
    
    def _generate_correlation_matrix(self,data,porcentaje = 0.1,m = 10000):
        """
        Genera la matrix de correlaciones para la 
        """
        
        m,n = np.shape(data)
        W = np.random.normal(0, 1, (m,n))
        rho_ini = np.empty([m,n])
        np.fill_diagonal(rho_ini, 1)
        for dim1 in range(n):
            for dim2 in range(n):
                if dim1 < dim2 and dim1 != dim2:
                    rho = np.corrcoef(W[:,dim1],W[:,dim2])[0,1]
                    if rho < 0:
                        l = -1
                        u = 0
                    else:
                        l = 0
                        u = 1
                    rho_estimado = self._rho_function(rho,W[:,dim1],W[:,dim2],m,dim1,dim2)
                    while np.absolute(rho_estimado - rho) > porcentaje*rho:
                        if rho_estimado > rho:
                            u = rho
                        else:
                            l = rho
                        r = (l+u)/2
                        rho_estimado = self._rho_function(rho,W[:,dim1],W[:,dim2],m,dim1,dim2)
                    rho_ini[dim1,dim2] = rho_estimado
                    rho_ini[dim2,dim1] = rho_estimado
        print(rho_ini)
        
        
    def _rho_function(self,rho,dimention1,dimention2,m,dim1,dim2):
        z = np.empty([m,2])
        z[:,0] = np.copy(dimention1)
        z[:,1] = dimention1 + dimention2*(np.sqrt(1-np.power(rho,2)))*dimention2
        x = np.empty([m,2])
        for func in [self.contenedor[dim1]['function']]:
            x[:,0] = func.ppf(q=norm.cdf(z[:,0]),loc=self.contenedor[dim1]['mu'],
                              scale=self.contenedor[dim1]['sigma'])
        for func in [self.contenedor[dim2]['function']]:
            x[:,1] = func.ppf(q=norm.cdf(z[:,1]),loc=self.contenedor[dim2]['a'],
                              scale=self.contenedor[dim2]['b'])
        rho_estimado = np.sum((x[:,0]-np.mean(x[:,0]))*(x[:,1]-np.mean(x[:,1])))/np.sqrt(((np.power(np.sum((x[:,0]-np.mean(x[:,0])))),2))*((np.power(np.sum((x[:,1]-np.mean(x[:,1]))),2))))
        print(rho_estimado)
        return rho_estimado
        
    def set_and_fit_data(self,data):
        """
        Recibe la data, calcula cual es la distribución que mejor queda para cada dimensión, y luego genera el
        modelo.
        """
        pass
    
    def _generate(self):
        self.R = np.corrcoef(M)
        self.L = np.linalg.cholesky(self.R) #lower
        self.functions = args
        self.arguments = kwargs
        
    def inverse_uniform(self,Zi,a,b):
        return ((b-a)*Zi-a)/b
    
    def generate(self,n):
        W = np.random.normal(0, 1, (self.m,n))
        Z = np.dot(self.L,W)
        X = np.empty([self.m, n])
        for i in range(self.m):
            for j in range(n):
                if self.functions[i]=='uniform':
                    X[i,j] = self.inverse_uniform(
                        norm.cdf(Z[i,j],self.arguments.get('a'),self.arguments.get('b')))
                elif self.functions[i] == 'normal':
                    X[i,j] = self.inverse_normal(
                        norm.cdf(Z[i,j]),self.arguments.get('mu'),
                        self.arguments.get('sigma'))
        return X
    
    def show_corr(self):
        print("Correlation between dimensions:", self.R)

In [41]:
M = np.random.randint(10,20,size=(500,2))
Prueba = Norta()
Prueba.set_marginal('normal',mu=15,sigma=1)
Prueba.set_marginal('uniform',a=10, b=20)
Prueba.set_data(M)

normal
Function successfully added

uniform
Function successfully added

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.8

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.88577478

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.88577478

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.88577478

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.88577478

...

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.88577478

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.88577478

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.88577478

3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.885774784559469e+28
3.88577478

KeyboardInterrupt: 

In [ ]:
Prueba.show_corr()
X = Prueba.generate(n=100)
plt.scatter(X[0,:],X[1,:])
plt.title("Plot of Dimension's relation")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()
print("Estimate Correlation:",np.corrcoef(X))